In [3]:
#Install & Import Libraries
!pip install transformers datasets -q

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from torch.optim import AdamW # Corrected import
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [4]:
#Load or Generate Dataset
def load_problem_data(dummy=True, path=None):
    if dummy:
        np.random.seed(42)
        n = 500
        texts = [
            "We're solving last-mile delivery for rural India.",
            "A mobile app to remind people to drink water.",
            "AI-powered tool for hospital inventory management.",
            "Helping students find internships faster.",
            "Blockchain loyalty rewards system for cafes."
        ]
        labels = ["Critical Need", "Nice to Have", "Critical Need", "Strong Want", "Nice to Have"]

        df = pd.DataFrame({
            'problem_statement': np.random.choice(texts, n),
            'label': np.random.choice(labels, n)
        })
        return df
    else:
        return pd.read_csv(path)

In [5]:
#Encode Labels and Tokenize
class ProblemDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [6]:
#Prepare Data & Tokenizer
df = load_problem_data()

label_encoder = LabelEncoder()
df['label_id'] = label_encoder.fit_transform(df['label'])  # e.g., Critical Need → 0, Strong Want → 1, etc.

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['problem_statement'], df['label_id'], test_size=0.2, random_state=42
)

train_dataset = ProblemDataset(train_texts.tolist(), train_labels.tolist(), tokenizer)
val_dataset = ProblemDataset(val_texts.tolist(), val_labels.tolist(), tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
#Build Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
#Train the Model (Basic Loop)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3
lr_scheduler = get_scheduler("linear", optimizer=optimizer,
                             num_warmup_steps=0,
                             num_training_steps=len(train_loader) * num_epochs)

model.train()
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}")
    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels)

        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

Epoch 1


  8%|▊         | 2/25 [02:00<23:02, 60.09s/it]


KeyboardInterrupt: 

In [ ]:
#Make Predictions and Map to Score
def predict_and_score(texts):
    model.eval()
    scores = []
    score_map = {
        "Critical Need": 1.0,
        "Strong Want": 0.7,
        "Nice to Have": 0.3
    }
    with torch.no_grad():
        for text in texts:
            encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
            input_ids = encoding['input_ids'].to(device)
            attention_mask = encoding['attention_mask'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            pred_id = torch.argmax(outputs.logits, dim=1).item()
            label = label_encoder.inverse_transform([pred_id])[0]
            score = score_map[label]
            scores.append(score)

    return scores